# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
csv_path = "weather_df"

# Read Purchasing File and store into Pandas data frame
city_weather_df = pd.read_csv(csv_path)
city_weather_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Wind Speed,Cloudiness
0,Ushuaia,AR,-54.80,-68.30,46.4,70,27.51,75
1,Brae,GB,60.40,-1.35,46.4,87,6.93,100
2,Bambous Virieux,MU,-20.34,57.76,78.8,78,8.05,40
3,Bredasdorp,ZA,-34.53,20.04,64.4,77,5.82,0
4,Riverton,US,40.52,-111.94,59.0,44,9.17,1


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Store latitude and longitude in locations
locations = city_weather_df[["Lat", "Lng"]]

# Fill NaN values and convert to float
humidity = city_weather_df["Humidity"].astype(float)

In [4]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [11]:
ideal_city_df = city_weather_df.copy()
ideal_city_df = ideal_city_df[ideal_city_df['Max Temp'] >= 70]
ideal_city_df = ideal_city_df[ideal_city_df['Max Temp'] <= 80]  
ideal_city_df = ideal_city_df[ideal_city_df['Cloudiness'] == 0]
ideal_city_df = ideal_city_df[ideal_city_df['Humidity'] < 40]
ideal_city_df

,City,Country,Lat,Lng,Max Temp,Humidity,Wind Speed,Cloudiness
251,Alegrete,BR,-29.78,-55.79,73.69,38,9.75,0
374,Auch,FR,43.65,0.58,71.60,24,10.29,0
495,Tura,IN,25.52,90.22,73.81,35,4.54,0
573,Tezu,IN,27.92,96.17,71.31,31,3.94,0


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [12]:
hotel_df = ideal_city_df
hotel_df['Hotel Name'] = ""
hotel_df

,City,Country,Lat,Lng,Max Temp,Humidity,Wind Speed,Cloudiness,Hotel Name
251,Alegrete,BR,-29.78,-55.79,73.69,38,9.75,0,
374,Auch,FR,43.65,0.58,71.60,24,10.29,0,
495,Tura,IN,25.52,90.22,73.81,35,4.54,0,
573,Tezu,IN,27.92,96.17,71.31,31,3.94,0,


In [13]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    'types': 'lodging',
    'radius': 5000,
    'key': g_key
}

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]

    params["location"] = f"{lat},{lng}"
    
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
  
    results = response["results"]
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    
    except (KeyError, IndexError):
        print("Missing field/result... skipping.") 

Retrieving Results for Index 251: Alegrete.
Closest hotel is Hotel Alegrete.
Retrieving Results for Index 374: Auch.
Closest hotel is Hôtel de France.
Retrieving Results for Index 495: Tura.
Closest hotel is Hotel RIKMAN Continental.
Retrieving Results for Index 573: Tezu.
Closest hotel is Circuit House.


In [14]:
hotel_df

,City,Country,Lat,Lng,Max Temp,Humidity,Wind Speed,Cloudiness,Hotel Name
251,Alegrete,BR,-29.78,-55.79,73.69,38,9.75,0,Hotel Alegrete
374,Auch,FR,43.65,0.58,71.60,24,10.29,0,Hôtel de France
495,Tura,IN,25.52,90.22,73.81,35,4.54,0,Hotel RIKMAN Continental
573,Tezu,IN,27.92,96.17,71.31,31,3.94,0,Circuit House


In [18]:
# NOTE: Do not change any of the code in this cell
locations = hotel_df[["Lat", "Lng"]]
markers = gmaps.marker_layer(locations)

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [21]:
# Create hotel symbol layer
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color = 'rgba(0, 0, 150, 0.4)', scale = 2,
    info_box_content = hotel_info
)
# Add the layer to the map
fig.add_layer(markers)
fig.add_layer(hotel_layer)
fig

Figure(layout=FigureLayout(height='420px'))